In [7]:
!rm /content/News_CSVs

In [8]:
from google.colab import drive
drive.mount("/content/drive", force_remount=True)
!ln -s "/content/drive/MyDrive/News_CSVs"

Mounted at /content/drive


In [13]:
import os
import string
import re
import nltk
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.5.0'
spark_version = 'spark-3.5.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://dlcdn.apache.org/spark/spark-3.5.0/spark-3.5.0-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:4 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:6 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease [18.1 kB]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:9 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [1,617 kB]
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/multiverse amd64 Packages [50.4 kB]
Get:13 http://archive.ubuntu.com/ubuntu jammy-update

In [14]:
os.system('ln -s /content/drive/MyDrive/News_CSVs/model.pkl model.pkl')

256

In [15]:
# Dependencies
import pandas as pd
import numpy as np
import nltk, string,re
from string import punctuation
from nltk.corpus import stopwords
from sklearn import metrics
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow as tf

from pyspark.sql import SparkSession
from pyspark.sql.functions import when, lit, col, to_date, rand, lower

from sklearn import model_selection
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import balanced_accuracy_score, confusion_matrix, classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from nltk.stem.wordnet import WordNetLemmatizer

nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('punkt')

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [24]:
#test data
test_news =["The mother and grandfather of five-month-old Palestinian baby Muhammad Hani al-Zahar held his dead body in front of a hospital following the resumption of hostilities in Gaza on the first day of December." ]

In [25]:
# Data save in Dataframe
news_dict = {'news': test_news}
df = pd.DataFrame(news_dict)

In [26]:
df.to_csv('news_data.csv', index=False)

In [27]:
# Load the trained model
loaded_model = pickle.load(open('model.pkl', 'rb'))

In [28]:
# Load the new dataset
new_data = pd.read_csv('news_data.csv')

In [29]:
# Preprocess the new dataset

# Define the function for text preprocessing
def preprocess_text(text):

    # Convert text to lowercase
    text = text.lower()
    # Remove URLs
    pattern_url = re.compile(r'(https?://\S+)|(www\.\S+)|(\S+\.\S+/\S+)')
    text = pattern_url.sub(r'', text)

    # Remove special characters
    pattern = r'[^\w\s]'
    text = re.sub(pattern, '', text)

    # Remove stop words
    stop_words = set(stopwords.words('english'))
    text = " ".join([word for word in text.split() if word not in stop_words])

    # Lemmatize text
    lemmatizer = WordNetLemmatizer()
    text = " ".join([lemmatizer.lemmatize(word) for word in nltk.word_tokenize(text)])

    # Remove tokens
    # Remove repeating characters
    text = re.sub(r'(\w)\1+', r'\1', text)

    # Remove tokens containing digits
    text = re.sub(r'\b\w*\d\w*\b', '', text)

    # Remove tokens containing underscore
    text = re.sub(r'\b\w*_\w*\b', '', text)

    # Remove tokens containing special characters
    text = re.sub(r'\b\w*[^\w\s]+\w*\b', '', text)

    # Remove tokens less than 2 characters
    text = re.sub(r'\b\w{1}\b', '', text)

    return text

 # Apply the text preprocessing function to the dataset
new_data['news'] = new_data['news'].apply(preprocess_text)
new_data

,news
0,mother grandfather fivemonthold palestinian ba...


In [30]:
# Define the tokenizer object
MAX_NUM_WORDS = 10000
tokenizer = Tokenizer(num_words=MAX_NUM_WORDS)
tokenizer.fit_on_texts(new_data['news'])

# Make predictions using the loaded model
MAX_SEQUENCE_LENGTH = 101
testX = tokenizer.texts_to_sequences(new_data['news'])
testX = pad_sequences(testX, maxlen=MAX_SEQUENCE_LENGTH)
predictions = loaded_model.predict(testX)

#Create a new DataFrame to store the predictions along with the original text data
df_test = pd.DataFrame({'test_sent': new_data['news'], 'prediction': predictions})
df_test['test_sent'] = df_test['test_sent'].apply(preprocess_text)

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(


In [31]:
#Classify the text data as “Fake” or “Real” based on the predicted probabilities
df_test['prediction'] = df_test['prediction'].apply(lambda x: "Fake" if x>=0.5 else "Real")
df_test

,test_sent,prediction
0,mother grandfather fivemonthold palestinian ba...,Real
